In [42]:
import pandas as pd
import requests 
from bs4 import BeautifulSoup 
import xmltodict 
import json
import xml.etree.ElementTree as et 

In [43]:
# http://openapi.gbis.go.kr/ws/rest/busrouteservice?serviceKey=1234567890&keyword=11
# return value :  String routeName:노선 번호

#Rest 접속 xml값받아오기 

bus_no = 112

In [53]:
service_url = "http://apis.data.go.kr/6410000/busrouteservice"
service_name = "/getBusRouteList?"
encoding_key = "%2BltohkyQC0eQUMVVaH5qwUi4FxaROssy0kpwzEdkqsFqedo%2FKlvT05Ap0svSUr2xQsOHd9%2FK2pXWpnH5N%2BmTcg%3D%3D" 
serviceKey = "serviceKey="+encoding_key 
keyword = f"&keyword={bus_no}"

#station_search_url = "http://apis.data.go.kr/6410000/busstationservice/getBusStationList?"+ f"serviceKey={serviceKey}" + "&keyword=전원아파트"

In [54]:
final_url = service_url + service_name + serviceKey + keyword

In [55]:
print(final_url)

http://apis.data.go.kr/6410000/busrouteservice/getBusRouteList?serviceKey=%2BltohkyQC0eQUMVVaH5qwUi4FxaROssy0kpwzEdkqsFqedo%2FKlvT05Ap0svSUr2xQsOHd9%2FK2pXWpnH5N%2BmTcg%3D%3D&keyword=112


In [56]:
bus_info_xml = requests.get(final_url)

In [ ]:
#msgBody -> busRouteList -> egionName(==수원) -> <routeId>200000049</routeId>  <routeName>112</routeName>

In [59]:
print(bus_info_xml.text)

<?xml version="1.0" encoding="UTF-8" standalone="yes"?><response><comMsgHeader/><msgHeader><queryTime>2023-09-23 14:42:01.672</queryTime><resultCode>0</resultCode><resultMessage>정상적으로 처리되었습니다.</resultMessage></msgHeader><msgBody><busRouteList><districtCd>2</districtCd><regionName>수원</regionName><routeId>200000049</routeId><routeName>112</routeName><routeTypeCd>13</routeTypeCd><routeTypeName>일반형시내버스</routeTypeName></busRouteList><busRouteList><districtCd>2</districtCd><regionName>남양주,서울,하남</regionName><routeId>222000045</routeId><routeName>112-1</routeName><routeTypeCd>13</routeTypeCd><routeTypeName>일반형시내버스</routeTypeName></busRouteList><busRouteList><districtCd>2</districtCd><regionName>서울,하남</regionName><routeId>227000005</routeId><routeName>112-5</routeName><routeTypeCd>13</routeTypeCd><routeTypeName>일반형시내버스</routeTypeName></busRouteList></msgBody></response>


In [60]:
#xmlt0dict 
content = bus_info_xml.content
dic = xmltodict.parse(content)
print(dic)

def xtod(xml_data):
    #contents 분리 
    content = xml_data.content 
    #dictionary 볂환 
    bus_route_dic = xmltodict.parse(content)
    return bus_route_dic

    

OrderedDict([('response', OrderedDict([('comMsgHeader', None), ('msgHeader', OrderedDict([('queryTime', '2023-09-23 14:42:01.672'), ('resultCode', '0'), ('resultMessage', '정상적으로 처리되었습니다.')])), ('msgBody', OrderedDict([('busRouteList', [OrderedDict([('districtCd', '2'), ('regionName', '수원'), ('routeId', '200000049'), ('routeName', '112'), ('routeTypeCd', '13'), ('routeTypeName', '일반형시내버스')]), OrderedDict([('districtCd', '2'), ('regionName', '남양주,서울,하남'), ('routeId', '222000045'), ('routeName', '112-1'), ('routeTypeCd', '13'), ('routeTypeName', '일반형시내버스')]), OrderedDict([('districtCd', '2'), ('regionName', '서울,하남'), ('routeId', '227000005'), ('routeName', '112-5'), ('routeTypeCd', '13'), ('routeTypeName', '일반형시내버스')])])]))]))])


In [61]:
jsonString = json.dumps(dic['response']['msgBody']['busRouteList'])
json_object = json.loads(jsonString)

#json -> 판다스 DF로 변환 

df = pd.DataFrame(json_object)


def make_df(dic_obj):
    jsonString = json.dumps(dic['response']['msgBody']['busRouteList'])
    json_object = json.loads(jsonString)
    df = pd.DataFrame(json_object)
    return df 


In [83]:
inSuwon= df['regionName'] == '수원'

In [95]:
route_Id = df[inSuwon]['routeId']
print(route_Id[0])

200000049


In [85]:
df.loc[:"regionName"]

,districtCd,regionName,routeId,routeName,routeTypeCd,routeTypeName
0,2,수원,200000049,112,13,일반형시내버스
1,2,"남양주,서울,하남",222000045,112-1,13,일반형시내버스
2,2,"서울,하남",227000005,112-5,13,일반형시내버스
